# Start coding and Enjoy the journey

## Install the repo

In [ ]:

#change branch
!git clone -b adaptive_reranking --single-branch --recursive https://github.com/Digre01/Visual-Place-Recognition-Project.git

## Install dependencies

In [ ]:
%cd Visual-Place-Recognition-Project/image-matching-models
!pip install -e .[all]

In [ ]:
!pip install faiss-cpu

## Download Datasets

In [ ]:
!python download_datasets.py

## Run your First VPR Evalutation



In [ ]:
#extract here retrival time

!python VPR-methods-evaluation/main.py \
--num_workers 4 \
--batch_size 32 \
--log_dir log_dir \
--method=cosplace --backbone=ResNet18 --descriptors_dimension=512 \
--image_size 512 512 \
--database_folder 'data/svox/images/test/gallery' \
--queries_folder 'data/svox/images/test/queries_sun' \
--num_preds_to_save 20 \
--recall_values 1 5 10 20 \
--faiss_metric "DP" \
--save_for_uncertainty 


## Run Image Matching on Retrieval Results



### Adaptive re-ranking

In [ ]:
##USE THIS THRESHOLDS 10 20 30 45 65

!python match_queries_preds.py \
--preds-dir '<path-to-predictions-folder>' \
--matcher 'superpoint-lg' \
--device 'cuda' \
--num-preds 20 \
--adaptive-threshold 15

In [ ]:
#Run this for the results
!python adaptive_reranking.py  \
    --preds-dir '/kaggle/working/Visual-Place-Recognition-Project/logs/log_dir/2026-01-15_09-57-58/preds' \
    --inliers-dir '/kaggle/working/Visual-Place-Recognition-Project/logs/log_dir/2026-01-15_09-57-58/preds_superpoint-lg' \
    --num-preds 20 

## Check Re-ranking Performance

In [ ]:
!python reranking.py \
--preds-dir '<path-to-predictions-folder>' \
--inliers-dir '<path-to-inliers-folder>' \
--num-preds 20 \
--recall-values 1 5 10 20

## Perform Uncertainty Evalutation

In [ ]:
!python -m vpr_uncertainty.eval \
--preds-dir '<path-to-predictions-folder>' \
--inliers-dir '<path-to-inliers-folder>' \
--z-data-path '<path-to-z-data-file>'   

In [ ]:
###############################

#DO NOT USE ITS NOT WORKING

################################

import subprocess
import re
import matplotlib.pyplot as plt
import os

# --- 1. CONFIGURATION ---
# Set the correct path to the folder produced by main.py
PREDS_DIR = 'logs/log_dir/2026-01-14_12-00-00/predictions' 
THRESHOLDS = [0, 10, 20, 30, 40, 60, 100] # Inlier thresholds to test
MATCHER = 'superpoint-lg'

recalls = []
savings = []

print(f"Starting adaptive analysis on {len(THRESHOLDS)} configurations...")

# --- 2. EXECUTION LOOP ---
for t in THRESHOLDS:
    print(f"Analyzing threshold T={t}...", end=" ")
    out_dir = f"inliers_adaptive_t{t}"
    
    # Run adaptive matching
    subprocess.run([
        "python", "match_queries_preds.py",
        "--preds-dir", PREDS_DIR,
        "--out-dir", out_dir,
        "--matcher", MATCHER,
        "--adaptive-threshold", str(t),
        "--num-preds", "20"
    ], capture_output=True)

    # Run re-ranking and capture logs
    res = subprocess.run([
        "python", "reranking.py",
        "--preds-dir", PREDS_DIR,
        "--inliers-dir", out_dir,
        "--num-preds", "20"
    ], capture_output=True, text=True).stdout
    
    # Extract data using Regex from strings printed by reranking.py
    r1_match = re.search(r"R@1: ([\d.]+)", res)
    s_match = re.search(r"Cost Savings: ([\d.]+)%", res)
    
    if r1_match and s_match:
        r1 = float(r1_match.group(1))
        s = float(s_match.group(1))
        recalls.append(r1)
        savings.append(s)
        print(f"Done! (R@1: {r1}%, Saving: {s}%)")
    else:
        print("Error extracting data from output.")

# --- 3. PLOT GENERATION ---

# Plot 1: Recall & Savings vs Threshold
fig, ax1 = plt.subplots(figsize=(10, 5))

ax1.set_xlabel('Inlier Threshold')
ax1.set_ylabel('Recall@1 (%)', color='tab:blue')
ax1.plot(THRESHOLDS, recalls, marker='o', color='tab:blue', label='Recall@1')
ax1.tick_params(axis='y', labelcolor='tab:blue')

ax2 = ax1.twinx()
ax2.set_ylabel('Cost Savings (%)', color='tab:green')
ax2.plot(THRESHOLDS, savings, marker='s', color='tab:green', label='Cost Savings')
ax2.tick_params(axis='y', labelcolor='tab:green')

plt.title('Impact of Threshold on Accuracy and Efficiency')
plt.grid(True, alpha=0.3)
plt.show()

# Plot 2: Trade-off Curve (Recall vs Cost Savings)
plt.figure(figsize=(8, 5))
plt.plot(savings, recalls, marker='D', color='tab:red', linestyle='--')
for i, t in enumerate(THRESHOLDS):
    plt.annotate(f"T={t}", (savings[i], recalls[i]), textcoords="offset points", xytext=(5,5))

plt.xlabel('Cost Savings (%)')
plt.ylabel('Recall@1 (%)')
plt.title('Trade-off Curve: Accuracy vs Efficiency')
plt.grid(True)
plt.show()